In [1]:
import pandas as pd

In [3]:
pd.read_csv("/home/fetchuser/ghostlab-fetch-experiment/pose_data_movement_50Hz1763088955.42.csv")

,x,y,z,alpha,beta,gamma


In [ ]:
pd.plotting